In [34]:
import os
import yaml

from azureml.core.conda_dependencies import CondaDependencies

from rsidatasciencetools.azureutils.azureconfig import AzureConfig

In [2]:
d = ('/mnt/c/Users/NickRoseveare/Documents/Work/UbuntuVM/code/DataScienceTools/'
     'dstools/azureutils/example_run/')

In [35]:
cfg = AzureConfig('./')
ws = cfg.Workspace

/home/nick/Work/code/DataScienceTools/dstools/config/baseconfig.py:154: UserWarning: for *config type: AzureConfig found more than one (json) *config file in this directory, overriding / replacing according to read-in order
  warn((f'for {self.base_str} type: {self.__class__.__name__} '


In [21]:
with open(os.path.join(d,'train-in-azure.yml'),'r') as f:
    yaml_config = yaml.safe_load(f)
strict_yaml_loc = os.path.join('./temp.yml')
envdict = {k:v for k,v in yaml_config.items() if k in CondaDependencies._VALID_YML_KEYS}
envdict['name'] = '-'.join([
    yaml_config['name'],yaml_config['env_name']])
with open(strict_yaml_loc,'w') as f:
    yaml.safe_dump(envdict, f)
conda_dependencies = CondaDependencies(
    conda_dependencies_file_path=strict_yaml_loc)
try:
    os.remove(strict_yaml_loc)
except FileNotFoundError:
    pass
del envdict

In [31]:
# Usually a good idea to choose specific version numbers
# so training is made on same packages as scoring

# create env yml file
conda_dependencies.add_pip_package(pip_package='debugpy')
conda_dependencies.add_pip_package(pip_package='azureml-defaults==1.0.83')

with open("./debugenv.yml","w") as f:
    f.write(conda_dependencies.serialize_to_string())
    # f.write('image: mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04')


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Model

model_name = 'sklearn-ridge-regr-diabetes'

env = Environment.from_conda_specification(name="env", file_path="./debugenv.yml")
env.docker.base_image = None
env.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
inference_config = InferenceConfig(entry_script="score.py", environment=env)
model = Model(workspace=ws, name=model_name)
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)  # Or show_output=False to hide the Docker build logs.
package.pull()

In [ ]:
# tag the docker image
#    docker tag my/path/to/image debug:1

In [ ]:
# to run in the command line:
#     docker run -it --name debug -p 8000:5001 -p 5678:5678 -v (/mnt/c/Users/NickRoseveare/Documents/Work/UbuntuVM/code/DataScienceTools/dstools/azureutils/example_run/score.py:/var/azureml-app/score.py debug:1 /bin/bash